### Optimizing Temporary Housing Deployment to Reduce U.S. Homelessness 
Nauman Sohani, Yannan Tuo, Charlie Nitschelm

In [7]:
# Load packages
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [8]:
# Generate distance matrix
%run ./optimization/coc-distance.ipynb
%whos

# print(coc_distance_matrix.head())

366
366
366
['AK_500', 'AK_501', 'AL_500', 'AL_501', 'AL_502', 'AL_503', 'AL_504', 'AL_505', 'AL_506', 'AL_507', 'AR_500', 'AR_501', 'AR_503', 'AR_505', 'AZ_500', 'AZ_501', 'AZ_502', 'CA_500', 'CA_501', 'CA_502', 'CA_503', 'CA_504', 'CA_505', 'CA_506', 'CA_507', 'CA_508', 'CA_509', 'CA_510', 'CA_511', 'CA_512', 'CA_513', 'CA_514', 'CA_515', 'CA_516', 'CA_517', 'CA_518', 'CA_519', 'CA_520', 'CA_521', 'CA_522', 'CA_524', 'CA_525', 'CA_526', 'CA_600', 'CA_601', 'CA_602', 'CA_603', 'CA_604', 'CA_606', 'CA_607', 'CA_608', 'CA_609', 'CA_611', 'CA_612', 'CA_613', 'CA_614', 'CO_500', 'CO_503', 'CO_504', 'CT_503', 'CT_505', 'DE_500', 'FL_500', 'FL_501', 'FL_502', 'FL_503', 'FL_504', 'FL_505', 'FL_506', 'FL_507', 'FL_508', 'FL_509', 'FL_510', 'FL_511', 'FL_512', 'FL_513', 'FL_514', 'FL_515', 'FL_517', 'FL_518', 'FL_519', 'FL_520', 'FL_600', 'FL_601', 'FL_602', 'FL_603', 'FL_604', 'FL_605', 'FL_606', 'GA_500', 'GA_501', 'GA_502', 'GA_503', 'GA_504', 'GA_505', 'GA_506', 'GA_507', 'GA_508', 'HI_500

In [9]:
# Set up Gurobi environment
env = gp.Env(empty=True)
env.setParam('OutputFlag', 0)
env.start()

# Initialize the model
m = gp.Model(env=env)

In [10]:
# High level variables + assumptions
# Total budget allocated for uplift units
TotalBudget =  50000000 # 130000000 # 50000 # 10000000 # budget: 85281049.54872459

# Uplift unit specification assumptions
# Uplift Factory Spec 
FactoryCost = factory_estimate
# FactoryCost = 10000 

# Uplift THU Spec 
THUCost = 5000

# Uplift Shipping Spec 
THUShippingBase = 100
THUShippingPerMile = 1

# Uplift Operational Spec 
THU_Factory_Limit = 999999999999999  # 15000 #1500
# THU_to_Factory_Max_Distance = 50

# Big M
M = 999999999999999999999

# Given no factory production limit, how much budget is needed for it to make sense to build second factory
# vary over budget


In [11]:
# City distance data
# print(coc_distance_matrix.head())
# city_distance = {
#                  "SF": {"SF": 0, "Boston": 3000, "NYC": 2900}, 
#                  "Boston": {"SF": 3000, "Boston": 0, "NYC": 215},
#                  "NYC": {"SF": 2900, "Boston": 215, "NYC": 0}
#                  }

In [12]:
# CoC Data/homeless population
# CoCPopulations = CoC_populations
# CoCPopulations = {"SF": 8300, "Boston": 6000, "NYC": 350000}
# CoCPopulations = {"SF": 9000, "Boston": 6000, "NYC": 15000}
# CoCPopulations = {"SF": 5000, "Boston": 5000, "NYC": 5000}


# Number of cities
NumCities = numCoCs

# Number of factory locations available (equivalent to cities)
# todo: rename var
NumFactories = numCoCs

# ordered dict for version < 3.7; assumed same order in version >= 3.7
indices = range(len(CoC_populations))
# CoCs = CoC_populations['CoC_Number'].values
indexToCityDict = dict(zip(indices, cocs))
def indexToCity(index):
    return indexToCityDict[index]

# print(indexToCityDict)

print(NumCities)
print(NumFactories)
print(len(FactoryCost))

366
366
366


In [13]:
# Decision variables
# How many homes to place per city
THU = m.addVars((t for t in range(0, NumCities)), lb=0, name="THUQuantityPerCity")

# Where to place factories (also # of factories)
Factory = m.addVars((t for t in range(0, NumFactories)), vtype=GRB.BINARY, name="FactoryLocations")

# How many THUs shipped from a given factory
THUShippedFromFactory = m.addMVar((NumCities, NumFactories), name="ClosestFactory", lb=0)
# City = m.addVars(((i, j) for i in range(0, NumCities) for j in range(0, NumFactories)), lb=0, ub = 1, name="ClosestFactory")


In [ ]:
# Constraints
# Budget constraint
# sum cost of all the THUs + 
# sum cost of all the factories + 
# sum cost of transportation of THUs from all the factories = # produced in city * closest factory * distance to factory * shipping cost per mile
budgetConstr = m.addConstr((sum(THU[cityIndex]*THUCost for cityIndex in range(0, NumCities)) + 
                              sum(FactoryCost.iloc[factory]['microhome_cost']*Factory[factory] for factory in range(NumFactories)) +
                              sum(THUShippedFromFactory[(cityIndex, factory)]*coc_distance_matrix.iloc[cityIndex, factory]*THUShippingPerMile 
                              for cityIndex in range(NumCities) for factory in range(NumFactories))) <= TotalBudget, name='BudgetConstr')

# Unhoused population (allocate no more than # unhoused)
popConstr = m.addConstrs(THU[cityIndex] <= CoC_populations.iloc[cityIndex] for cityIndex in range(0, NumCities))

# sum of THUs shipped needs to equal THUs in location
THUTotal = m.addConstrs(sum(THUShippedFromFactory[cityIndex][factoryIndex] for factoryIndex in range(NumFactories)) == THU[cityIndex] for cityIndex in range(NumCities))

# enforce no THU production if factory not selected
THUProdatFactory = m.addConstrs(THUShippedFromFactory[cityIndex][factoryIndex] <= M*Factory[factoryIndex] for cityIndex in range(NumCities) for factoryIndex in range(NumFactories))
FactoryLimit = m.addConstrs(sum(THUShippedFromFactory[cityIndex][factoryIndex] for cityIndex in range(NumCities)) <= THU_Factory_Limit for factoryIndex in range(NumFactories))

# Todo: max and min # THUs per factory

# There must be at least one factory in total
factoryConstr = m.addConstr(sum(Factory[factory] for factory in range(NumFactories)) >= 1)


In [39]:
# Objective function
# Maximize number of housing units provided for unhoused populations (ie; minimize unhoused individiuals)
m.setObjective(sum(THU[city] for city in range(0, NumCities)), gp.GRB.MAXIMIZE)


In [40]:
# Update and write the model
m.update() # Update model parameters
# m.write("uplift.lp") # Write model to file

# Solve
m.optimize()

# Check model is not infeasible
if m.status == GRB.INFEASIBLE:
    print("model is infeasible")
print("\nObjective value: ", "%.2f" % m.getAttr("ObjVal"))



Objective value:  9810.77


In [41]:
# Print solution
print("THUs:")
for i in THU:
    # if THU[i].x > 0:
        print('%s: %g' % (indexToCity(i), THU[i].x))

print("Factories:")
for i in Factory:
    # if THU[i].x > 0:
        print('%s: %g' % (indexToCity(i), Factory[i].x))

print("Shipping from factories:")
for i in range(NumCities):
    for j in range(NumFactories):
        if THUShippedFromFactory[i][j].x> 0:
            print('source city: %s, factory city: %s, value: %g' % (indexToCity(i), indexToCity(j), THUShippedFromFactory[i][j].x))

THUExpense = sum(THU[cityIndex].x*THUCost for cityIndex in range(0, NumCities))
FactoryExpense = sum(FactoryCost.iloc[factory]['microhome_cost']*Factory[factory].x for factory in range(NumFactories))
shipCost = sum(THUShippedFromFactory[cityIndex][factory].x*coc_distance_matrix.iloc[cityIndex, factory]*THUShippingPerMile for cityIndex in range(NumCities) for factory in range(NumFactories))
print("Expected max budget: " + str(TotalBudget) + " actual budget: " + str(THUExpense + FactoryExpense + shipCost))
print("THU cost: ", THUExpense)
print("Factory cost: " + str(FactoryExpense))
print("THU ship cost: " + str(shipCost))

THUs:
AK_500: 0
AK_501: 0
AL_500: 0
AL_501: 0
AL_502: 0
AL_503: 0
AL_504: 0
AL_505: 0
AL_506: 0
AL_507: 0
AZ_500: 1713.77
AZ_501: 2179
AZ_502: 5918
Factories:
AK_500: -0
AK_501: -0
AL_500: -0
AL_501: -0
AL_502: -0
AL_503: -0
AL_504: -0
AL_505: -0
AL_506: -0
AL_507: -0
AZ_500: 0
AZ_501: -0
AZ_502: 1
Shipping from factories:
source city: AZ_500, factory city: AZ_502, value: 1713.77
source city: AZ_501, factory city: AZ_502, value: 2179
source city: AZ_502, factory city: AZ_502, value: 5918
Expected max budget: 50000000 actual budget: 50000000.0
THU cost:  49053846.43879988
Factory cost: 565386.0
THU ship cost: 380767.561200122


In [75]:
print(CoC_populations)

            Overall_Homeless
CoC_Number                  
AK_500                1023.0
AK_501                 761.0
AL_500                1329.0
AL_501                 598.0
AL_502                 209.0
AL_503                 536.0
AL_504                 490.0
AL_505                 438.0
AL_506                 245.0
AL_507                 716.0
AZ_500                2398.0
AZ_501                2179.0
AZ_502                5918.0


#### Analysis of optimization results
beep boop